In [2]:
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def mse_function(answer, prediction):
    mse = ((answer - prediction)**2).mean(axis=0)
    for i in range(len(answer.shape)-1):
        mse = mse.mean(axis=0)
    return mse

Data to test on

In [ ]:
# n = 10000
# time_span = 5
# c = 100
# time_step = 0.1
# integration_time_step = 0.01
# number_timesteps_predict = 2
# std = 0

# Load the observations
# name = f"{n}_{time_span}_{str(time_step).replace('.', '')}_{str(integration_time_step).replace('.', '')}_{number_timesteps_predict}_{str(std).replace('.','')}"
# dictionary = np.load(f"data_dictionaries/data_{name}.npy", allow_pickle=True).item()
# observations = dictionary["observations"]
# c = 25
# predictions = dictionary[c]
# print(dictionary)

# # Calculate the mean squared error between the true observation and the imperfect model's prediction
# finalmse = mse_function(observations_scaled[:,-1,:], predictions_scaled[:,-1,:])
# print(finalmse)

In [3]:
# Variables relating to the data you want to load
n = 10000
time_span = 5
time_step = 0.1
integration_time_step = 0.01
number_timesteps_predict = 5
std = 0.01

In [ ]:
name = f"{n}_{time_span}_{str(time_step).replace('.', '')}_{str(integration_time_step).replace('.', '')}_{number_timesteps_predict}_{str(std).replace('.', '')}"
MLonly_model = keras.models.load_model(f'saved_models/timesteps_{number_timesteps_predict}/MLonly_{name}.keras')

In [ ]:
c_array = [0.1, 1, 25, 50, 75,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000]

name = f"{n}_{time_span}_{str(time_step).replace('.', '')}_{str(integration_time_step).replace('.', '')}_{number_timesteps_predict}_{str(std).replace('.', '')}"
dictionary = np.load(f"data_dictionaries/data_{name}.npy", allow_pickle=True).item()
# dictionary['mse_imperfect_predictions'] = {}
# dictionary['physical_loss'] = {}
# dictionary['physical_error_loss'] = {}
# dictionary['physical_two_stage_training_loss'] = {}
test_observations = dictionary["observations"]

for i, c in enumerate(c_array):
    print(c)
    test_predictions = dictionary[c]

    # Calculate the mean squared error between the true observation and the imperfect model's prediction
    finalmse = mse_function(test_observations[:,-1,:], test_predictions)
    print(f"mse of imperfect predictions = {finalmse}")
    dictionary['mse_imperfect_predictions'][c] = finalmse

    # #### Plot!
    # print(f'c={c}')
    # for i in range(2):
    #     plt.plot(test_observations[i,-5:,1], test_observations[i,-5:,2])
    #     plt.scatter(test_predictions[i,1], test_predictions[i,2])
    #     plt.show()
    # #####

    if c != np.inf:
        print('c not infinity') 
        test_X = {"input_ob": test_observations[:,:-number_timesteps_predict,:], "input_pred": test_predictions}
    
        # print(str(c).replace('.', ''))
        # loaded_model = keras.models.load_model(f"saved_models/timesteps_{number_timesteps_predict}/physical_c{str(c).replace('.', '')}_{name}.keras")
        # print('loaded!')
        # loss_physical = loaded_model.evaluate(x = test_X, y = test_observations[:,-1,:])
        # print('evaluated!')
        # print(f"loss = {loss_physical}")
        # dictionary['physical_loss'][c] = loss_physical
    
        
        loaded_model = keras.models.load_model(f"saved_models/timesteps_{number_timesteps_predict}/physical_error_c{str(c).replace('.', '')}_{name}.keras")
        print('loaded!')
        loss_physical_error = loaded_model.evaluate(x = test_X, y = test_observations[:,-1,:] - test_predictions )
        print('evaluated!')
        print(f"loss = {loss_physical_error}")
        dictionary['physical_error_loss'][c] = loss_physical_error
    
        # loaded_model = keras.models.load_model(f"saved_models/timesteps_{number_timesteps_predict}/physical_two_stage_training_c{str(c).replace('.', '')}_{name}.keras")
        # print('loaded!')
        # loss_physical_two_stage_training = loaded_model.evaluate(x = test_X, y = test_observations[:,-1,:] )
        # print('evaluated!')
        # print(f"loss = {loss_physical_two_stage_training}")
        # dictionary['physical_two_stage_training_loss'][c] = loss_physical_two_stage_training

MLonly_loss = MLonly_model.evaluate(x = test_observations[:,:-number_timesteps_predict,:], y = test_observations[:,-1,:])
print(f'ML only loss = {MLonly_loss}')
dictionary['MLonly_loss'] = MLonly_loss
np.save(f"data_dictionaries/data_{name}", dictionary)

In [5]:
c_array = [0.1, 1, 25, 50, 75,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000]
name = f"{n}_{time_span}_{str(time_step).replace('.', '')}_{str(integration_time_step).replace('.', '')}_{number_timesteps_predict}_{str(std).replace('.', '')}"
dictionary = np.load(f"data_dictionaries/data_{name}.npy", allow_pickle=True).item()
finalmse = np.zeros(len(c_array))
loss_physical = np.zeros(len(c_array))
loss_physical_error = np.zeros(len(c_array))
# loss_physical_two_stage_training =  np.zeros(5)
MLonly_loss = dictionary['MLonly_loss']
# mse_perfect_model_prediction = dictionary['mse_imperfect_predictions'][np.inf]

for i, c in enumerate(c_array):
    finalmse[i] = dictionary['mse_imperfect_predictions'][c]
    loss_physical[i] = dictionary['physical_loss'][c]
    loss_physical_error[i] = dictionary['physical_error_loss'][c]
# for i, c in enumerate([25, 100, 500, 900, 1000]):
#     loss_physical_two_stage_training[i] = dictionary['physical_two_stage_training_loss'][c]

plt.plot(c_array,finalmse,'-', label = 'imperfect model', color = 'g', )
plt.plot(c_array, loss_physical, '-', label = 'ML physical model', color = 'r')
plt.plot(c_array, loss_physical_error, '-', label = 'ML physical error model', color = 'purple')
# plt.plot([25, 100, 500, 900, 1000], loss_physical_two_stage_training, '-', label = 'ML physical two stage training model', color = 'orange')
plt.hlines(MLonly_loss, 0,1000, label = 'ML only model', color = 'b')
# plt.hlines(mse_perfect_model_prediction, 0,1000, label = 'perfect model', color = 'b')
plt.hlines(3*10**(-8), 0,1000, linestyle = '-.', label = 'perfect model', color = 'b')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('c')
plt.ylabel('mse')
plt.grid(which='major', color='black', linestyle='-', linewidth = 0.2)
# plt.grid(which='minor', color='grey', linestyle='-')
plt.legend()
plt.title('mean squared error of models against c')
plt.savefig(f'mse_vs_c_timesteps{number_timesteps_predict}_{name}')
plt.show()

plt.scatter(finalmse, loss_physical, 'x', label = 'ML physical model', color = 'r')
plt.scatter(finalmse, loss_physical_error, '+', label = 'ML physical error model', color = 'purple')
# plt.plot(c_array, loss_physical_two_stage_training, '-', label = 'ML physical two stage training model', color = 'orange')
plt.hlines(MLonly_loss, 0,1, label = 'ML only model', color = 'b')
# plt.hlines(mse_perfect_model_prediction, 0,1000, label = 'perfect model', color = 'b')
# plt.hlines(3*10**(-8), 0,1, linestyle = '-.', label = 'perfect model', color = 'b')
plt.plot(linspace(0,1,2), linspace(0,1,2), '--', label = 'y=x', color = 'b')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('imperfect model mse')
plt.ylabel('mse')
plt.grid(which='major', color='black', linestyle='-', linewidth = 0.2)
# plt.grid(which='minor', color='grey', linestyle='-')
plt.legend()
plt.title('mse of ML models against mse of the imperfect physical model')
plt.savefig(f'mse_vs_mse_timesteps{number_timesteps_predict}_{name}')


FileNotFoundError: [Errno 2] No such file or directory: 'data_dictionaries/data_10000_5_01_001_5_001.npy'

In [ ]:
print('hi')